Start this first
```bash
$ litex_server --uart --uart-port /dev/ttyUSB0 --uart-baudrate 1152000
```

In [6]:
%pylab notebook
from struct import pack, unpack
import time
from scipy.signal import *
import sys
sys.path.append("../")
from common import *

Populating the interactive namespace from numpy and matplotlib


/home/michael/miniconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['csd', 'square', 'correlate', 'convolve', 'bartlett', 'blackman', 'detrend', 'exponential', 'hanning', 'hamming', 'step', 'kaiser']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [9]:
r.close()

NameError: name 'r' is not defined

In [8]:
r = conLitexServer()

Connected to Port 1234


timeout: timed out

In [448]:
r.regs.acq_trig_level.read()

32768

# Setup SPI

In [449]:
ltc_spi = LTC_SPI(r)

# Dump LTC2175 registers

In [450]:
for i in range(5):
    print("{:2x}: {:02x}".format(i, ltc_spi.get_ltc_reg(i)))

 0: 00
 1: 00
 2: 00
 3: 00
 4: 00


In [451]:
ltc_spi.set_ltc_reg(0, 0x80)   # Software reset

In [ ]:
ltc_spi.set_ltc_reg(1, 0x10)   # Sleep mode

In [ ]:
ltc_spi.set_ltc_reg(1, 0x00)   # Waky waky

# Read and align clock data

In [452]:
def print_frm():
    rVal = r.regs.lvds_frame_peek.read()
    print(r.regs.lvds_idelay_value.read())
    print("0x{0:02X} 0b{0:08b}".format(rVal))
print_frm()

15
0x0F 0b00001111


### bitslip until we get 0xF0 on frame_peek

In [384]:
r.regs.lvds_bitslip_csr.write(1)
print_frm()

15
0xF0 0b11110000


### IDELAY adjustments

In [415]:
r.regs.lvds_idelay_inc.write(1)
print_frm()

22
0xE1 0b11100001


In [422]:
r.regs.lvds_idelay_dec.write(1)
print_frm()

15
0xF0 0b11110000


do a bitslip until we get 0x0F

# Map data lanes
Check that each bit in the test-pattern end up in the right place in the output word

In [423]:
for ch in range(4):
    print("Channel {:d}".format(ch))
    for i in range(14):
        tpValue = 1 << i
        ltc_spi.setTp(tpValue)
        rVal = getattr(r.regs, 'lvds_data_peek{:d}'.format(ch)).read()
        print("{:016b} {:016b}".format(tpValue, rVal))    

Channel 0
0000000000000001 0000000000000100
0000000000000010 0000000000001000
0000000000000100 0000000000010000
0000000000001000 0000000000100000
0000000000010000 0000000001000000
0000000000100000 0000000010000000
0000000001000000 0000000100000000
0000000010000000 0000001000000000
0000000100000000 0000010000000000
0000001000000000 0000100000000000
0000010000000000 0001000000000000
0000100000000000 0010000000000000
0001000000000000 0100000000000000
0010000000000000 1000000000000000
Channel 1
0000000000000001 0000000000000100
0000000000000010 0000000000001000
0000000000000100 0000000000010000
0000000000001000 0000000000100000
0000000000010000 0000000001000000
0000000000100000 0000000010000000
0000000001000000 0000000100000000
0000000010000000 0000001000000000
0000000100000000 0000010000000000
0000001000000000 0000100000000000
0000010000000000 0001000000000000
0000100000000000 0010000000000000
0001000000000000 0100000000000000
0010000000000000 1000000000000000
Channel 2
0000000000000001 0

# Read ADC sample

In [424]:
ltc_spi.set_ltc_reg(3, 0)   # Test pattern off
ltc_spi.set_ltc_reg(1, 0)   # Randomizer off

In [429]:
# Lowest 2 bits are always zero in 16 bit LVDS mode
# 12 bit ADC, so next 2 bits are also zero
rVal = r.regs.lvds_data_peek3.read() >> 4
print("0x{0:03x} 0b{0:012b}".format(rVal))
# Decode offset binary mode
print(rVal - (1 << 11))

0x7fc 0b011111111100
-4


# Read randomized ADC sample

In [ ]:
ltc_spi.set_ltc_reg(1, (1<<6))   # Randomizer on

In [ ]:
rVal = r.regs.lvds_data_peek.read() >> 4
print("0x{0:03x} 0b{0:012b}".format(rVal))
# XOR LSB with any other bit
if rVal & 0x01:
    rVal ^= 0xFFE
# Decode offset binary mode
print(rVal - (1 << 11))

In [ ]:
r.close()